In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
import RigidBodyDynamics.Spatial
import LCPSim
import CartPoles
import Nets
using JuMP
using Gurobi
using Mosek
using ProfileView
import ConditionalJuMP
using BenchmarkTools
using StaticArrays
using Base.Test

In [ ]:
cartpole = CartPoles.CartPole()
mechanism = cartpole.mechanism

In [ ]:
x0 = MechanismState{Float64}(mechanism)
configuration(x0) .= [0.1, 0.2]
x = LCPSim.StateRecord(x0)
xnext = LCPSim.LinearizedState{Variable}(x0)
u = zeros(2)
@benchmark LCPSim.update($x, $xnext, $u, $(cartpole.environment), 0.04, m) setup=(m=Model())

In [ ]:
x = LCPSim.StateRecord(x0)
xnext = LCPSim.LinearizedState{Variable}(x0)
u = zeros(2)
model = Model(solver=GurobiSolver())
f() = LCPSim.optimize(x0, cartpole.environment, 0.04, 200, model)
# f() = for i in 1:100; LCPSim.update(x, xnext, u, cartpole.environment, 0.04, model); end

f()
Profile.clear()
@time @profile f();
ProfileView.view()

In [ ]:
x0 = MechanismState{Float64}(mechanism)
solver = GurobiSolver(Gurobi.Env(), OutputFlag=0)
g() = LCPSim.simulate(x0, x -> zeros(num_velocities(x0)), cartpole.environment, 0.04, 1000, solver)
g()
Profile.clear()
@time @profile g()
ProfileView.view()